In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import wandb
from pathlib import Path

from views_forecasts.extensions import *
from utils import fetch_data, transform_data, get_config_path, get_config_from_path, retrain_transformed_sweep, evaluate
from utils_map import plot_cm_map, plot_pgm_map

import os
os.environ['WANDB_SILENT'] = 'true'


In [2]:
level = 'cm'
config_path = Path('./my_config_cm')

In [3]:
transforms = ['raw', 'log', 'normalize', 'standardize']
Datasets_transformed = {}
para_transformed = {}
qslist, Datasets = fetch_data(level)
for t in transforms:
    Datasets_transformed[t], para_transformed[t] = transform_data(Datasets, t, level, by_group=True)

Fetching query sets
 .     Fetching datasets
 .     

In [16]:
def train():
    run = wandb.init()
    model_paras = [para for para in run.config.keys() if para !='transform']
    run_name = f'transform_{run.config["transform"]}'
    for para in model_paras:
        run_name += f'_{para}_{run.config[para]}'
    wandb.run.name = run_name
    # print(run_name)
    
    wandb.config.update(common_config, allow_val_change=True)
    wandb.config.update(model_config, allow_val_change=True)
    
    retrain_transformed_sweep(Datasets_transformed, model_paras)
    evaluate('calib', para_transformed, by_group=True)
    run.finish()

In [15]:
common_config_path, wandb_config_path, model_config_path, sweep_config_path = get_config_path(config_path)
common_config = get_config_from_path(common_config_path, 'common')
wandb_config = get_config_from_path(wandb_config_path, 'wandb')

In [12]:
for sweep_file in sweep_config_path.iterdir():
    if sweep_file.is_file():
        model_file = model_config_path / sweep_file.name
        if not model_file.is_file():
            raise FileNotFoundError(f'The corresponding model configuration file {model_file} does not exist.')

        sweep_config = get_config_from_path(sweep_file, 'sweep')
        model_config = get_config_from_path(model_file, 'model')
    
        if sweep_file.stem.split('_')[-2] == 'hurdle':
            continue # Currently Hurdle models are not supported
        model = sweep_file.stem.split('_')[-1]
        # sweep_paras = PARA_DICT[model]
        sweep_id = wandb.sweep(sweep_config, project=wandb_config['project'],
                               entity=wandb_config['entity'])
        wandb.agent(sweep_id, function=train)
        print(f'Finish sweeping over model {sweep_file.stem}')
    break

Create sweep with ID: 7wx39ouu
Sweep URL: https://wandb.ai/model-development-and-deployment/test001/sweeps/7wx39ouu
['learning_rate', 'n_estimators', 'n_jobs', 'transform']
['learning_rate', 'n_estimators', 'n_jobs', 'transform']
Finish sweeping over model fatalities003_pgm_baseline_lgbm


In [11]:
stepcols = ['ged_sb_dep']
steps = [*range(1, 36 + 1, 1)]
for step in steps:
    stepcols.append('step_pred_' + str(step))
run_id = 'Fatalities003'
name = 'cm_fatalities003_nl_baseline_rf_calib_transform_raw_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'

In [14]:
name = 'cm_fatalities003_nl_baseline_rf_calib_transform_raw_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df.describe()

pr_56_cm_fatalities003_nl_baseline_rf_calib_transform_raw_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5.parquet


,ged_sb_dep,step_pred_1,step_pred_2,step_pred_3,step_pred_4,step_pred_5,step_pred_6,step_pred_7,step_pred_8,step_pred_9,...,step_pred_27,step_pred_28,step_pred_29,step_pred_30,step_pred_31,step_pred_32,step_pred_33,step_pred_34,step_pred_35,step_pred_36
count,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,...,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000
mean,43.984293,2.763453,2.719436,2.763168,1.894036,1.928245,1.827149,1.621647,1.837690,1.649706,...,1.242454,1.237369,1.164332,1.202674,1.216925,1.166410,1.115325,1.138484,1.148242,1.224143
std,478.269169,3.672684,3.542995,3.772312,2.644334,2.367929,2.027407,1.869207,2.093655,1.758595,...,1.432982,1.447862,1.432327,1.397935,1.329776,1.451369,1.308437,1.410775,1.367916,1.252939
min,0.000000,0.480216,0.480745,0.482173,0.490586,0.491545,0.485932,0.496379,0.491325,0.493941,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.748876,0.772803,0.729018,0.711627,0.791510,0.739163,0.752162,0.738416,0.753256,...,0.650071,0.647873,0.608584,0.632843,0.649850,0.600785,0.527729,0.576196,0.598334,0.699598
50%,0.000000,2.634222,2.419861,2.716885,1.371664,1.396560,1.437981,1.095765,1.448651,1.228742,...,0.745319,0.729310,0.657638,0.716428,0.750603,0.643519,0.587826,0.602538,0.630868,0.793501
75%,0.000000,3.265092,3.173449,3.235476,1.622635,1.757818,1.741029,1.335304,1.620372,1.421709,...,0.967016,0.993814,0.858438,0.967920,0.985358,0.851137,0.967359,0.895460,0.911281,1.062088
max,19000.000000,46.281792,47.852463,49.803467,33.822624,28.254168,28.297398,17.863476,23.644480,18.066242,...,20.867840,22.295361,28.512068,24.516321,16.908724,20.122360,16.604708,29.344843,22.985306,15.077058


In [22]:
name = 'cm_fatalities003_nl_baseline_rf_calib_transform_log_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df = np.exp(df) - 1
df

pr_56_cm_fatalities003_nl_baseline_rf_calib_transform_log_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5.parquet


ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                      
397      1                  0.0     0.613933     0.616953     0.615754   
         2                  0.0     0.628708     0.627454     0.626578   
         3                  0.0     0.620750     0.618255     0.618701   
         4                  0.0     0.623216     0.613123     0.613084   
         5                  0.0     0.628965     0.627593     0.626561   
...                         ...          ...          ...          ...   
444      242                0.0     0.627109     0.635177     0.629947   
         243                0.0     0.646165     0.636574     0.633543   
         244                0.0     0.614472     0.614351     0.612793   
         245                0.0     0.694161     0.798478     0.900139   
         246               14.0     0.887280     0.842067     0.759033   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
397      1              0.614390     0.616320     0.616750     0.614932   
         2              0.623116     0.627264     0.625510     0.623309   
         3              0.618201     0.619136     0.617050     0.617961   
         4              0.613172     0.614843     0.615930     0.614272   
         5              0.623811     0.628646     0.624390     0.624241   
...                          ...          ...          ...          ...   
444      242            0.625829     0.630717     0.627193     0.628726   
         243            0.626536     0.643574     0.631771     0.638303   
         244            0.612464     0.612587     0.613666     0.612700   
         245            0.834527     0.846576     0.877188     0.880774   
         246            0.913548     0.898830     0.806334     0.801293   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
397      1              0.620469     0.617564  ...      0.617548   
         2              0.636764     0.630672  ...      0.621717   
         3              0.624933     0.621965  ...      0.616367   
         4              0.615191     0.614701  ...      0.624097   
         5              0.636951     0.631354  ...      0.622985   
...                          ...          ...  ...           ...   
444      242            0.621645     0.630045  ...      0.640423   
         243            0.638083     0.628973  ...      0.641926   
         244            0.614742     0.613690  ...      0.620062   
         245            0.885611     0.882882  ...      0.781598   
         246            0.829002     0.765058  ...      0.868647   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
397      1               0.616562      0.615784      0.617495      0.619100   
         2               0.615270      0.616696      0.617528      0.622300   
         3               0.612105      0.616021      0.612847      0.615287   
         4               0.625719      0.633003      0.623588      0.631281   
         5               0.615514      0.616201      0.617878      0.622994   
...                           ...           ...           ...           ...   
444      242             0.655568      0.629049      0.635070      0.639029   
         243             0.659257      0.626271      0.643949      0.646233   
         244             0.620757      0.621573      0.620810      0.616627   
         245             0.763405      0.765723      0.832082      0.844601   
         246             0.854042      0.821235      0.806700      0.776165   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                                           
397      1 

In [17]:
para_transformed['standardize']['baseline003']

,mean_val,std_val
country_id,,
1,0.000000,0.000000
2,0.000000,0.000000
3,0.045775,1.096525
4,0.191315,3.660668
5,0.000000,0.000000
...,...,...
248,0.000000,NaN
250,0.000000,NaN
252,0.000000,NaN


In [20]:
def standardize_retransform(x, mean_val, std_val):
    return x * std_val + mean_val
name = 'cm_fatalities003_nl_baseline_rf_calib_transform_standardize_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'
df_para_model = para_transformed['standardize']['baseline003']
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df = df.apply(lambda row: standardize_retransform(row, df_para_model['mean_val'].loc[row.name[1]], df_para_model['std_val'].loc[row.name[1]]), axis=1)
df

pr_56_cm_fatalities003_nl_baseline_rf_calib_transform_standardize_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5.parquet


ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                      
397      1                  0.0     0.000000     0.000000     0.000000   
         2                  0.0     0.000000     0.000000     0.000000   
         3                  0.0     0.568336     0.568134     0.568391   
         4                  0.0     1.916907     1.916631     1.907969   
         5                  0.0     0.000000     0.000000     0.000000   
...                         ...          ...          ...          ...   
444      242                0.0     0.544947     0.548019     0.551224   
         243                0.0     3.179936     3.166709     3.163791   
         244                0.0     0.287449     0.287058     0.287274   
         245                0.0   169.287914   171.018399   172.103673   
         246               14.0    40.774208    40.938232    39.934496   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
397      1              0.000000     0.000000     0.000000     0.000000   
         2              0.000000     0.000000     0.000000     0.000000   
         3              0.566591     0.566438     0.566815     0.566233   
         4              1.906660     1.917911     1.909212     1.909986   
         5              0.000000     0.000000     0.000000     0.000000   
...                          ...          ...          ...          ...   
444      242            0.547349     0.547731     0.547068     0.548666   
         243            3.158991     3.160678     3.169139     3.157281   
         244            0.287270     0.287344     0.287443     0.287834   
         245          170.782257   170.767298   170.756183   170.928340   
         246           41.167741    41.285561    40.292174    40.108534   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
397      1              0.000000     0.000000  ...      0.000000   
         2              0.000000     0.000000  ...      0.000000   
         3              0.567021     0.568354  ...      0.566616   
         4              1.912638     1.906618  ...      1.916026   
         5              0.000000     0.000000  ...      0.000000   
...                          ...          ...  ...           ...   
444      242            0.548416     0.547005  ...      0.554154   
         243            3.165691     3.173427  ...      3.174238   
         244            0.288015     0.287467  ...      0.288000   
         245          170.894070   170.555161  ...    169.847792   
         246           40.244101    40.322599  ...     40.612325   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
397      1               0.000000      0.000000      0.000000      0.000000   
         2               0.000000      0.000000      0.000000      0.000000   
         3               0.566050      0.565800      0.567109      0.566194   
         4               1.918890      1.932348      1.915606      1.918482   
         5               0.000000      0.000000      0.000000      0.000000   
...                           ...           ...           ...           ...   
444      242             0.553195      0.550166      0.550887      0.552065   
         243             3.182089      3.155188      3.166767      3.164034   
         244             0.287278      0.287829      0.287174      0.287844   
         245           169.446313    169.744989    170.488995    170.782842   
         246            40.423381     40.802971     40.113731     40.172517   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                                           
397      1 

In [21]:
def normalize_retransform(x, min_val, max_val, b=1, a=0):
    return (x - a) / (b - a) * (max_val - min_val) + min_val
name = 'cm_fatalities003_nl_baseline_rf_calib_transform_normalize_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'
df_para_model = para_transformed['normalize']['baseline003']
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df = df.apply(lambda row: normalize_retransform(row, df_para_model['min_val'].loc[row.name[1]], df_para_model['max_val'].loc[row.name[1]]), axis=1)
df


pr_56_cm_fatalities003_nl_baseline_rf_calib_transform_normalize_n_estimators_100_n_jobs_15_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5.parquet


ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                      
397      1                  0.0     0.000000     0.000000     0.000000   
         2                  0.0     0.000000     0.000000     0.000000   
         3                  0.0    14.746427    14.736396    14.736063   
         4                  0.0    44.668164    44.661583    44.659625   
         5                  0.0     0.000000     0.000000     0.000000   
...                         ...          ...          ...          ...   
444      242                0.0    11.884318    11.887568    11.892138   
         243                0.0    82.752975    82.744921    82.723121   
         244                0.0     5.701192     5.700891     5.701339   
         245                0.0   283.673317   284.429160   285.377931   
         246               14.0   406.370638   406.428091   405.319159   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
397      1              0.000000     0.000000     0.000000     0.000000   
         2              0.000000     0.000000     0.000000     0.000000   
         3             14.731823    14.731092    14.732048    14.728331   
         4             44.662051    44.671018    44.675338    44.661264   
         5              0.000000     0.000000     0.000000     0.000000   
...                          ...          ...          ...          ...   
444      242           11.880030    11.885156    11.880813    11.884962   
         243           82.693921    82.734260    82.716271    82.747084   
         244            5.701179     5.701680     5.700988     5.701654   
         245          284.684418   284.766448   284.755489   284.891159   
         246          407.016908   407.037503   406.100954   405.724470   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
397      1              0.000000     0.000000  ...      0.000000   
         2              0.000000     0.000000  ...      0.000000   
         3             14.731538    14.732114  ...     14.729359   
         4             44.658174    44.672436  ...     44.673901   
         5              0.000000     0.000000  ...      0.000000   
...                          ...          ...  ...           ...   
444      242           11.896341    11.889368  ...     11.896967   
         243           82.766997    82.727093  ...     82.729873   
         244            5.701597     5.701646  ...      5.701873   
         245          284.817942   284.527486  ...    283.939055   
         246          406.027871   405.754869  ...    406.343229   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
397      1               0.000000      0.000000      0.000000      0.000000   
         2               0.000000      0.000000      0.000000      0.000000   
         3              14.728168     14.728981     14.729588     14.728914   
         4              44.694856     44.690306     44.684804     44.699713   
         5               0.000000      0.000000      0.000000      0.000000   
...                           ...           ...           ...           ...   
444      242            11.899932     11.880910     11.889268     11.886983   
         243            82.769180     82.717168     82.746119     82.770429   
         244             5.701480      5.701562      5.701800      5.701632   
         245           283.795121    283.869650    284.467958    284.630210   
         246           405.932927    406.510319    405.687004    405.615593   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                                           
397      1 

In [113]:
stepcols = ['ged_sb_dep']
steps = [*range(1, 36 + 1, 1)]
for step in steps:
    stepcols.append('step_pred_' + str(step))
run_id = 'Fatalities003'
name = 'cm_fatalities003_nl_topics_rf_calib_transform_raw_n_estimators_250_n_jobs_12_learning_rate_0.05_max_depth_12_min_child_weight_12_subsample_0.5_colsample_bytree_0.5'
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df

ged_sb_dep  step_pred_1  step_pred_2  step_pred_3  \
month_id country_id                                                      
397      1                  0.0     1.888033     1.536575     2.049761   
         2                  0.0     3.313491     2.442976     3.095902   
         3                  0.0     1.746300     1.800268     1.744825   
         4                  0.0     0.487359     0.500316     0.511034   
         5                  0.0     3.312054     2.441470     3.093398   
...                         ...          ...          ...          ...   
444      242                0.0     4.505231     5.732021     4.511056   
         243                0.0     0.564945     0.647136     0.593240   
         244                0.0     0.670239     1.084977     0.824759   
         245                0.0     5.004318     6.232212     8.742626   
         246               14.0     9.258842     6.313660     5.660348   

                     step_pred_4  step_pred_5  step_pred_6  step_pred_7  \
month_id country_id                                                       
397      1              1.194238     1.080280     1.230281     1.012267   
         2              1.791524     1.584468     1.561841     1.353563   
         3              1.028135     1.015283     1.214884     0.834816   
         4              0.510376     0.575729     0.510639     0.576125   
         5              1.789339     1.576807     1.559876     1.353323   
...                          ...          ...          ...          ...   
444      242           11.456407     1.927837     1.705806     2.175097   
         243            0.794530     0.965720     0.944105     0.807292   
         244            0.710583     1.426395     1.099138     1.305220   
         245            7.999347     5.930683     6.081397     5.129094   
         246            9.425748     6.468810     4.595462     5.049151   

                     step_pred_8  step_pred_9  ...  step_pred_27  \
month_id country_id                            ...                 
397      1              0.979016     1.124231  ...      0.605951   
         2              1.191040     1.440145  ...      0.726233   
         3              0.719306     0.796419  ...      1.161782   
         4              0.510816     0.504026  ...      0.665250   
         5              1.190878     1.439366  ...      0.723306   
...                          ...          ...  ...           ...   
444      242            7.634606     8.995864  ...      9.225223   
         243            1.222221     1.019295  ...      1.075368   
         244            1.311780     0.932088  ...      2.428319   
         245            5.645001     6.278614  ...      3.930102   
         246            5.514345     3.922381  ...      4.013035   

                     step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id country_id                                                           
397      1               0.616939      0.591293      0.696502      0.823638   
         2               0.770196      0.647228      0.721609      0.706091   
         3               1.024357      0.661682      0.779543      0.571230   
         4               0.683974      0.645012      0.484309      1.083846   
         5               0.757622      0.646705      0.720935      0.699926   
...                           ...           ...           ...           ...   
444      242             3.546402      3.188511      5.113416      3.803534   
         243             1.527027      0.485269      0.516639      0.520443   
         244             3.952767      0.813094      1.311723      0.843666   
         245             2.930245      3.637518      4.189796      4.103630   
         246             4.528634      4.591595      4.488104      4.181655   

                     step_pred_32  step_pred_33  step_pred_34  step_pred_35  \
month_id country_id                                                           
397      1 

In [114]:
df.describe()

,ged_sb_dep,step_pred_1,step_pred_2,step_pred_3,step_pred_4,step_pred_5,step_pred_6,step_pred_7,step_pred_8,step_pred_9,...,step_pred_27,step_pred_28,step_pred_29,step_pred_30,step_pred_31,step_pred_32,step_pred_33,step_pred_34,step_pred_35,step_pred_36
count,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,...,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000,9168.000000
mean,43.984293,3.064313,2.938524,3.024721,2.463671,2.344805,2.241646,2.080354,2.044865,2.113336,...,1.541634,1.638868,1.574594,1.621234,1.630235,1.545550,1.448616,1.550858,1.503650,1.700572
std,478.269169,4.013247,3.939282,3.012343,2.652986,2.608635,2.326226,2.094101,2.217072,2.182770,...,1.607832,1.771869,1.728240,1.659796,1.791969,1.802667,1.751915,1.778156,1.798335,2.024134
min,0.000000,0.476367,0.476422,0.476632,0.476452,0.476629,0.476453,0.476443,0.476490,0.476624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.141073,1.428083,1.226387,1.061084,1.030640,1.074640,0.995747,0.931747,0.949853,...,0.719530,0.752138,0.645578,0.720144,0.699926,0.572176,0.512930,0.581869,0.579295,0.768986
50%,0.000000,2.967103,2.444391,3.093398,1.789339,1.576807,1.629165,1.357512,1.201662,1.439366,...,0.785038,0.801496,0.758299,0.817740,0.767103,0.699220,0.666647,0.680501,0.691359,0.833531
75%,0.000000,3.312054,2.837250,3.346230,2.211347,2.121895,2.075090,1.941592,2.056416,2.010623,...,1.746570,1.930422,1.836017,1.928235,1.854694,1.792060,1.655463,1.842366,1.660575,1.698188
max,19000.000000,57.567913,58.776665,34.229748,25.622738,30.410690,36.324074,19.378551,22.624584,20.225647,...,19.464668,20.670267,17.030479,17.005161,19.832689,28.565922,27.920639,24.754251,28.713552,22.110678


In [147]:
months = df.index.levels[0].tolist()
step_preds = [f'step_pred_{i}' for i in range(1, 37)]
wandb.init(project='test', entity='model-development-and-deployment')
for month in [399, 400]:
    for step in ['step_pred_1']:
        fig = plot_cm_map(df, month, step)
        
        wandb.log({f'month_{month}_step_{step}': wandb.Image(fig)})
        
wandb.finish()

In [2]:
level = 'pgm'
transforms = ['raw', 'log', 'normalize', 'standardize']
qslist, Datasets = fetch_data(level)

Fetching query sets
 .     
A dataset with 8 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 19 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 29 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 24 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 23 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 30 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 8 columns, with data between t 1 and 852. (13110 units)
 .     
A dataset with 11 columns, with data between t 1 and 852. (13110 units)
Fetching datasets
 .     

In [23]:
Datasets_transformed = {}
para_transformed = {}
for t in transforms:
    Datasets_transformed[t], para_transformed[t] = transform_data(Datasets, t, level, by_group=True)

In [26]:
config_path = Path('./my_config_pgm')
common_config_path, wandb_config_path, model_config_path, sweep_config_path = get_config_path(config_path)
common_config = get_config_from_path(common_config_path, 'common')
wandb_config = get_config_from_path(wandb_config_path, 'wandb')

In [27]:
model_name = 'fatalities003_pgm_baseline_lgbm'
for sweep_file in sweep_config_path.iterdir():
    if sweep_file.is_file():
        # Skip if a specific model name is provided and it doesn't match the file
        model_name_from_file = sweep_file.stem
        if model_name and model_name != model_name_from_file:
            continue
            
        model_file = model_config_path / sweep_file.name
        if not model_file.is_file():
            raise FileNotFoundError(f'The corresponding model configuration file {model_file} does not exist.')

        sweep_config = get_config_from_path(sweep_file, 'sweep')
        model_config = get_config_from_path(model_file, 'model')
    
        if sweep_file.stem.split('_')[-2] == 'hurdle':
            continue # Currently Hurdle models are not supported
        model = sweep_file.stem.split('_')[-1]
        # sweep_paras = PARA_DICT[model]
        sweep_id = wandb.sweep(sweep_config, project=wandb_config['project'],
                               entity=wandb_config['entity'])
        wandb.agent(sweep_id, function=train)
        print(f'Finish sweeping over model {sweep_file.stem}')
    break
    

Create sweep with ID: 40divblj
Sweep URL: https://wandb.ai/model-development-and-deployment/pgm_example_1/sweeps/40divblj
transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12
{'learning_rate': 0.05, 'n_estimators': 100, 'n_jobs': 12}
Training model fatalities003_pgm_baseline_lgbm
Calibration partition (log)
 * == Performing a run: "pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12" == * 
Training model(s)...
Storing "pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12"
Getting predictions
pr_56_pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12.parquet
pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12 , run Fatalities003 does not exist, predicting
Test partition (log)
 * == Performing a run: "pgm_fatalities003_pgm_baseline_lgbm_test_transform_log_learning_rate_0.05_n_estimators

transform_normalize_learning_rate_0.05_n_estimators_100_n_jobs_12
{'learning_rate': 0.05, 'n_estimators': 100, 'n_jobs': 12}
Training model fatalities003_pgm_baseline_lgbm
Calibration partition (normalize)
 * == Performing a run: "pgm_fatalities003_pgm_baseline_lgbm_calib_transform_normalize_learning_rate_0.05_n_estimators_100_n_jobs_12" == * 
Training model(s)...
Storing "pgm_fatalities003_pgm_baseline_lgbm_calib_transform_normalize_learning_rate_0.05_n_estimators_100_n_jobs_12"
Getting predictions
pr_56_pgm_fatalities003_pgm_baseline_lgbm_calib_transform_normalize_learning_rate_0.05_n_estimators_100_n_jobs_12.parquet
pgm_fatalities003_pgm_baseline_lgbm_calib_transform_normalize_learning_rate_0.05_n_estimators_100_n_jobs_12 , run Fatalities003 does not exist, predicting
Finish sweeping over model fatalities003_pgm_baseline_lgbm


In [5]:
stepcols = ['ged_sb_dep']
steps = [*range(1, 36 + 1, 1)]
for step in steps:
    stepcols.append('step_pred_' + str(step))
run_id = 'Fatalities003'
name = 'pgm_fatalities003_pgm_baseline_lgbm_calib_transform_raw_learning_rate_0.05_n_estimators_100_n_jobs_12'
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df.describe()

pr_56_pgm_fatalities003_pgm_baseline_lgbm_calib_transform_raw_learning_rate_0.05_n_estimators_100_n_jobs_12.parquet


,ged_sb_dep,step_pred_1,step_pred_2,step_pred_3,step_pred_4,step_pred_5,step_pred_6,step_pred_7,step_pred_8,step_pred_9,...,step_pred_27,step_pred_28,step_pred_29,step_pred_30,step_pred_31,step_pred_32,step_pred_33,step_pred_34,step_pred_35,step_pred_36
count,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,...,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000
mean,0.357143,0.362960,0.235796,0.200281,0.192233,0.207696,0.253360,0.260690,0.217874,0.209900,...,0.085503,0.115182,0.102013,0.067152,0.109287,0.095474,0.082626,0.070612,0.074306,0.075961
std,10.742290,22.604209,9.727253,7.633099,6.278958,7.467056,10.196756,11.820635,7.514221,7.488593,...,3.642885,6.255992,6.586327,3.511985,5.003973,5.118448,3.472790,3.366107,3.609370,3.346155
min,0.000000,-369.145596,-239.012891,-124.988114,-323.638512,-198.734169,-123.406325,-115.332386,-135.203092,-101.517689,...,-197.290655,-145.531940,-206.145965,-386.248113,-369.163407,-249.280290,-290.583883,-219.790766,-197.321224,-355.121646
25%,0.000000,0.017658,0.019247,0.024417,0.025745,0.021106,0.024519,0.024257,0.025342,0.029282,...,0.008430,0.010251,0.012131,0.008328,0.014634,0.014106,0.013854,0.015003,0.015476,0.017671
50%,0.000000,0.017658,0.019247,0.024417,0.025745,0.021106,0.024519,0.024257,0.025342,0.029282,...,0.008430,0.010251,0.012395,0.008328,0.014634,0.014106,0.013854,0.015003,0.015476,0.017671
75%,0.000000,0.017658,0.019247,0.024417,0.025745,0.024515,0.024519,0.024257,0.025342,0.029282,...,0.013225,0.030053,0.014238,0.012387,0.024532,0.028845,0.017388,0.022591,0.017477,0.022242
max,1580.000000,3393.240804,3724.499363,2443.864943,1968.052512,1593.697184,1491.184652,1583.798864,1334.298983,2667.634904,...,696.297475,2566.186750,2621.295037,1203.775368,1397.033959,2085.250237,879.143073,1096.635328,1179.015076,619.578114


In [6]:
stepcols = ['ged_sb_dep']
steps = [*range(1, 36 + 1, 1)]
for step in steps:
    stepcols.append('step_pred_' + str(step))
run_id = 'Fatalities003'
name = 'pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12'
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df = np.exp(df) - 1
df.describe()

pr_56_pgm_fatalities003_pgm_baseline_lgbm_calib_transform_log_learning_rate_0.05_n_estimators_100_n_jobs_12.parquet


,ged_sb_dep,step_pred_1,step_pred_2,step_pred_3,step_pred_4,step_pred_5,step_pred_6,step_pred_7,step_pred_8,step_pred_9,...,step_pred_27,step_pred_28,step_pred_29,step_pred_30,step_pred_31,step_pred_32,step_pred_33,step_pred_34,step_pred_35,step_pred_36
count,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,...,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000,629280.000000
mean,0.357143,0.025410,0.022007,0.022355,0.020502,0.017919,0.017827,0.018912,0.016739,0.016003,...,0.011237,0.010115,0.010272,0.009740,0.010019,0.009432,0.009757,0.009439,0.008788,0.008548
std,10.742290,0.370579,0.320609,0.323944,0.296664,0.244225,0.267098,0.309219,0.237498,0.243263,...,0.203521,0.155079,0.167599,0.141783,0.165464,0.141041,0.173581,0.152443,0.134684,0.115434
min,0.000000,-0.075677,-0.042418,-0.069909,-0.026511,-0.139463,-0.036578,-0.061792,-0.052306,-0.070387,...,-0.018436,-0.001362,-0.045369,-0.029317,-0.074989,-0.052877,-0.017900,-0.042569,-0.059149,-0.034537
25%,0.000000,0.000629,0.000654,0.000694,0.000698,0.000737,0.000735,0.000778,0.000755,0.000747,...,0.000782,0.000783,0.000716,0.000719,0.000746,0.000750,0.000750,0.000789,0.000731,0.000730
50%,0.000000,0.000629,0.000654,0.000694,0.000698,0.000737,0.000735,0.000778,0.000755,0.000795,...,0.001086,0.001145,0.001064,0.001069,0.001074,0.001078,0.001098,0.001127,0.001113,0.001074
75%,0.000000,0.000629,0.000654,0.000741,0.000821,0.000807,0.000885,0.000941,0.000930,0.000932,...,0.001426,0.001505,0.001441,0.001555,0.001477,0.001455,0.001437,0.001489,0.001493,0.001542
max,1580.000000,36.395334,45.864467,29.982219,36.719954,28.361323,31.178444,29.255667,34.050488,41.508575,...,22.562600,19.041801,17.869934,13.054167,16.279341,13.031511,24.287855,21.757464,23.061903,12.259959


In [7]:
def standardize_retransform(x, mean_val, std_val):
    return x * std_val + mean_val
name = 'pgm_fatalities003_pgm_baseline_lgbm_calib_transform_standardize_learning_rate_0.05_n_estimators_100_n_jobs_12'
df_para_model = para_transformed['standardize']['baseline003']
df = pd.DataFrame.forecasts.read_store(run=run_id, name=name).replace([np.inf, -np.inf], 0)[stepcols]
df = df.apply(lambda row: standardize_retransform(row, df_para_model['mean_val'].loc[row.name[1]], df_para_model['std_val'].loc[row.name[1]]), axis=1)
df

NameError: name 'para_transformed' is not defined

In [4]:
months = df.index.levels[0].tolist()
step_preds = [f'step_pred_{i}' for i in range(1, 37)]
wandb.init(project='test', entity='model-development-and-deployment')
for month in [397]:
    for step in step_preds:
        fig = plot_pgm_map(df, month, step)
        
        wandb.log({f'month_{month}_{step}': wandb.Image(fig)})
        
wandb.finish()